# Load in extracted features

In [64]:
# open the every file in a directory and print the first line of each file
import os
import sys
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import BayesianRidge
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder


In [69]:
# load in the features
features_train = np.array([])
for file in os.listdir('features_train'):
    with open('features_train/' + file) as f:
        df = pd.read_csv(f)
        features_train = np.column_stack((features_train, df.values)) if features_train.size else df.values

# load the labels
train = pd.read_csv('train.csv')
y_train = train['Genre']
label_encoder = LabelEncoder()
# normalize the data
features_train = preprocessing.scale(features_train)

In [75]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

# Define the models
models = [
    ('Logistic Regression', LogisticRegression(max_iter = 150)),
    ('Random Forest', RandomForestClassifier(max_depth=2, random_state=0)),
    ('SVM', SVC())
]

# Function to find the best model
def find_best_model(x_train, y_train, models):
    best_score = 0
    best_model = None

    # Iterate over the models
    for name, model in models:
        # Perform cross-validation
        scores = cross_val_score(model, x_train, y_train, cv=5)
        # Calculate the mean score
        mean_score = scores.mean()

        # If the score is better than the current best, update the best score and model
        if mean_score > best_score:
            best_score = mean_score
            best_model = name

    return best_model, best_score

# Use the function to find the best model
best_model, best_score = find_best_model(features_train, y_train, models)
print(f"Best model: {best_model} with score: {best_score}")

Best model: Logistic Regression with score: 0.6570990566037735


In [76]:

logreg = LogisticRegression(max_iter=150)
logreg.fit(features_train, y_train)
predictions = logreg.predict(features_train)
# Calculate the accuracy
accuracy = accuracy_score(y_train, predictions)
print("Logistic Regression accuracy:", accuracy)

bayesian = BayesianRidge()
bayesian.fit(features_train, y_train)
predictions = bayesian.predict(features_train)
# Calculate the R-squared score
r2 = r2_score(y_train, predictions)
print("BayesianRidge R-squared:", r2)

Logistic Regression accuracy: 0.7396745932415519


ValueError: could not convert string to float: 'pop'

In [45]:
features_test = np.array([])
for file in os.listdir('features_test'):
    with open('features_test/' + file) as f:
        df = pd.read_csv(f)
        features_test = np.column_stack((features_test, df.values)) if features_test.size else df.values

# scale the test data
features_test = preprocessing.scale(features_test)

# run the model on the test data
predictions = logreg.predict(features_test)

# save the predictions to a csv file
df = pd.DataFrame({'ID': os.listdir('test'), 'genre': predictions})
df.to_csv('predictions.csv', index=False)